In [10]:
import pandas as pd
import numpy as np
import sys
import os
sys.path.append(os.path.abspath(os.path.join(os.getcwd(), '../../')))



df = pd.read_csv(os.path.join(os.getcwd(), "../../data/total_4am.csv"), index_col=0)


In [11]:
import pandas as pd
from performance import get_performance

def various_ma(interval_list, df):

    original_df = df.copy()  # Make a copy of the original DataFrame

    for interval in interval_list:
        df = original_df.copy()  # Create a fresh copy for each interval

        df[f"{interval}_ma"] = df['close'].rolling(window=interval).mean()

        # Implement RSI strategy for long positions only
        df['signal'] = 0  # Default to no position
        for i in range(interval, len(df)):
            # Buy condition
            if df.loc[i, 'close'] >= df.loc[i, f'{interval}_ma'] and df.loc[i-1, 'close'] < df.loc[i-1, f'{interval}_ma']:
                df.loc[i, 'signal'] = 1
            # Sell condition
            elif df.loc[i, 'close'] < df.loc[i, f'{interval}_ma'] and df.loc[i-1, 'close'] >= df.loc[i-1, f'{interval}_ma']:
                df.loc[i, 'signal'] = -1

        # Manage positions
        df['position'] = 0
        holding_position = False

        for i in range(1, len(df)):
            if df.loc[i, 'signal'] == 1 and not holding_position:
                df.loc[i, 'position'] = 1
                holding_position = True
            elif df.loc[i, 'signal'] == -1 and holding_position:
                df.loc[i, 'position'] = 0
                holding_position = False
            else:
                df.loc[i, 'position'] = df.loc[i-1, 'position']

        # Calculate the strategy returns (only when in a long position)
        df['strategy_returns'] = df['position'].shift(1) * df['close'].pct_change()
        df['strategy_returns2'] = df['strategy_returns']

        for i in range(1, len(df)):
            buy_price = df.loc[i-1, "close"]
            buy_price_copy = buy_price
            sell_price = df.loc[i, "close"]
            sell_price_copy = sell_price
            if df.loc[i-1, 'position'] == 1 and df.loc[i-1, 'signal'] == 1:
                # df.loc[i, 'strategy_returns2'] = (df.loc[i,'position'])/(df.loc[i-1, 'position'] * 1.002) -1
                buy_price = df.loc[i-1, "close"] * 1.002
            if df.loc[i, 'position'] == 0 and df.loc[i-1, 'position'] != 0:
                # df.loc[i, 'strategy_returns2'] = (df.loc[i,'position'] * 0.998)/(df.loc[i-1, 'position']) -1
                sell_price = df.loc[i, "close"] * 0.998
            
            if buy_price == buy_price_copy and sell_price == sell_price_copy:
                continue


            df.loc[i, "strategy_returns2"] = sell_price/buy_price - 1

        # Calculate the cumulative returns
        df['cumulative_returns'] = (1 + df['strategy_returns']).cumprod()
        df['cumulative_returns2'] = (1 + df['strategy_returns2']).cumprod()

        # Calculate the benchmark cumulative returns (buy and hold strategy)
        df['benchmark_returns'] = (1 + df['close'].pct_change()).cumprod()

        # Save the results to a CSV file
        # df.to_csv(f"../../results/ma/ma{(interval)}.csv")

        # Get performance metrics and plot the results
        get_performance(df=df, title=f"ma_{interval}")

        
    df = original_df.copy()  # Create a fresh copy for each interval


In [12]:
interval_list = [5, 10, 20, 50, 90, 120, 200]
various_ma(interval_list=interval_list, df=df)

======Investment Summary======
Strategy        : ma_5
total_return    : 284.08
cagr            : 48.99
mdd             : 44.87
total_return_w_fee : 112.48
cagr_w_fee      : 25.02
mdd_w_fee       : 55.89
investing_days  : 1232
======Investment Summary======
Strategy        : ma_10
total_return    : 187.53
cagr            : 37.63
mdd             : 38.32
total_return_w_fee : 99.0
cagr_w_fee      : 23.13
mdd_w_fee       : 46.48
investing_days  : 1207
======Investment Summary======
Strategy        : ma_20
total_return    : 171.94
cagr            : 35.43
mdd             : 43.81
total_return_w_fee : 113.92
cagr_w_fee      : 25.93
mdd_w_fee       : 49.87
investing_days  : 1204
======Investment Summary======
Strategy        : ma_50
total_return    : 269.83
cagr            : 53.09
mdd             : 42.29
total_return_w_fee : 234.64
cagr_w_fee      : 48.18
mdd_w_fee       : 45.33
investing_days  : 1121
======Investment Summary======
Strategy        : ma_90
total_return    : 134.5
cagr            